In [47]:
from collections import UserDict
import os
import zipfile
import json
from collections import UserDict

def tuple2str(var):
    from copy import deepcopy
    
    if not isinstance(var,(dict,UserDict)):
        return var
    
    newvar={}
    for key in var:
        if isinstance(key,tuple):  # either a board or board and player
            newtuple=[]
            for v in key:
                try:
                    len(v)
                    newtuple.append(tuple(v))
                except TypeError:
                    newtuple.append(v)

            newvar[str(tuple(newtuple))]=tuple2str(var[key])
        else:
            newvar[key]=tuple2str(var[key])
            
        
    return newvar

def str2table(var):
    from copy import deepcopy
    
    if not isinstance(var,dict):
        return var
    
    newvar=Table()
    for key in var:
        if (isinstance(key,str) or isinstance(key,unicode)) and key.startswith('(') and key.endswith(')'): # this is a tuple
            newkey=eval(key)
            newvar[newkey]=str2table(var[key])
        else:
            try:
                newkey=int(key)
            except ValueError:
                newkey=key
                
            newvar[newkey]=str2table(var[key])
            
        
    return newvar


def make_immutable(var):
    from copy import deepcopy
    
    try:
        var=var.immutable()
    except AttributeError:
        var=deepcopy(var)

    if isinstance(var,tuple):
        var=list(var)
    
    if isinstance(var,list):
        for i in range(len(var)):
            var[i]=make_immutable(var[i])
        return tuple(var)
    else:
        return var
        
from Game import Board
def state2key(state):
    if isinstance(state,Board):
        return tuple(state)
    elif isinstance(state,int):
        return state

    # can be a tuple or list

    return tuple(state2key(_) for _ in state)


class Table(dict):

    def __init__(self, other=None,**kwargs):
        
        if other:
            # Doesn't do keyword args
            if isinstance(other, dict):
                for k,v in list(other.items()):
                    k=make_immutable(k)
                    dict.__setitem__(self, k, v)
            else:
                for k,v in other:
                    k=make_immutable(k)
                    dict.__setitem__(self, k.lower(), v)

        if kwargs:
            for k,v in kwargs:
                k=make_immutable(k)
                dict.__setitem__(self, k.lower(), v)
            

    def max(self):
        s=[]
        for key in self.keys():
            s.append(self[key])

        return max(s)

    def argmax(self):
        
        s=[]
        for key in self.keys():
            s.append(self[key])

        argmax=max(zip(s, range(len(s))))[1]
        return argmax

    def min(self):
        s=[]
        for key in self.keys():
            s.append(self[key])

        return min(s)

    def __getitem__(self, key):
        key=make_immutable(key)
        return dict.__getitem__(self, key)

    def __setitem__(self, key, value):
        key=make_immutable(key)
        dict.__setitem__(self, key, value)

    def __contains__(self, key):
        key=make_immutable(key)
        try:
            value=dict.__contains__(self, key)
        except TypeError:
            print("Key is %s" % str(key))
            raise
        return value

    def has_key(self, key):
        key=make_immutable(key)
        return dict.has_key(self, key)

    def get(self, key, def_val=None):
        key=make_immutable(key)
        return dict.get(self, key, def_val)

    def setdefault(self, key, def_val=None):
        key=make_immutable(key)
        return dict.setdefault(self, key, def_val)

    def update(self, other):
        for k,v in list(other.items()):
            k=make_immutable(k)
            dict.__setitem__(self, k.lower(), v)

    def fromkeys(self, iterable, value=None):
        d = Dict()
        for k in iterable:
            k=make_immutable(k)
            
            dict.__setitem__(d, k, value)
        return d

    def pop(self, key, def_val=None):
        key=make_immutable(key)
        
        return dict.pop(self, key, def_val)
    
    def save(self,filename):
        SaveTable(self,filename)
        
    def load(self,filename):
    
        obj=LoadTable(filename)
        
        for key in obj:
            self[key]=obj[key]
        

def SaveTable(obj, filename='_memory_.json'):
    """Saves an object to disk
    
    Example:  Save([1,2,3])
    """
    
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'w', compression=zipfile.ZIP_DEFLATED) as f:
            f.writestr(filename[:-4],json.dumps(tuple2str(obj),sort_keys=True, indent=4))
    else:
        with open(filename, 'w') as f:
            json.dump(tuple2str(obj),f, sort_keys=True, indent=4,)

def LoadTable(filename='_memory_.json',handle_exist=True):
    """Loads an object from disk

    Example:  a=Load()
    """
    if handle_exist:
        if not os.path.exists(filename):
            T=Table()
            SaveTable(T,filename)
            return T

    if '.zip' in filename:
        with zipfile.ZipFile(filename, 'r') as f:
            data = f.read(filename[:-4])
            obj = json.loads(data)
    else:
        with open(filename,'r') as f:
            obj = json.load(f)

            
    obj=str2table(obj)
        
        
    return obj



In [52]:
import ijson
from Game.tables import make_immutable,str2table

class SmallTable(object):

    def __init__(self,filename):
        self.filename=filename

    def __getitem__(self, key):
        key=make_immutable(key)
        with open(self.filename, "rb") as f:
            for record in ijson.items(f, str(key)):
                return str2table(record)

        raise KeyError

    def __contains__(self, key):
        keyi=make_immutable(key)
        try:
            value=self[keyi]
            return True
        except KeyError:
            print(key)
            print(keyi.__repr__())
            return False



In [53]:
T=LoadTable("TTT Q2 Table.json")
T2=SmallTable("TTT Q2 Table.json")

In [54]:
from Game import Board
state=Board(3,3)
state[7]=1
state[8]=0
state

 0  0  0 
 0  0  0 
 0  1  0 

In [60]:
T2[state]

{0: Decimal('0.5118205736205067'),
 1: Decimal('0.5437698866979421'),
 2: Decimal('0.23025027055509165'),
 3: Decimal('0.3264742882318505'),
 4: Decimal('0.684579799793494'),
 5: Decimal('0.5145740211663807'),
 6: Decimal('0.5443245958716773'),
 8: Decimal('0.4312807581062598')}

In [59]:
from Game import top_choice
top_choice(T2[state])

4

In [56]:
state in T2

True

In [23]:
ijson.items?

Signature: ijson.items(source, prefix, map_type=None, buf_size=65536, **config)
Docstring: <no docstring>
File:      ~/venvs/work/lib/python3.12/site-packages/ijson/common.py
Type:      function

In [42]:
import ijson
import json
filename="TTT Q1 Table.json"
with open(filename, "rb") as f:
    for record in ijson.items(f, '(0, 0, 0, 0, 0, 0, 1, 0, 2)'):
        print(record)
        print(str2table(record))
        break

{'0': Decimal('0.884997849248777'), '1': Decimal('0.7555575661211674'), '2': Decimal('0.8424417020807933'), '3': Decimal('0.8299227675383533'), '4': Decimal('0.5455808777476829'), '5': Decimal('0.6590824262654658'), '7': Decimal('0.12939579972652365')}
{0: Decimal('0.884997849248777'), 1: Decimal('0.7555575661211674'), 2: Decimal('0.8424417020807933'), 3: Decimal('0.8299227675383533'), 4: Decimal('0.5455808777476829'), 5: Decimal('0.6590824262654658'), 7: Decimal('0.12939579972652365')}


In [18]:
record

NameError: name 'record' is not defined

In [19]:
with open(filename,'r') as f:
    obj = json.load(f)


In [40]:
with open(filename, "rb") as f:
    parser = ijson.parse(f)
    for prefix, event, value in parser:
        print(prefix, event, value)
        break

 start_map None


In [41]:
ijson.parse?

Signature: ijson.parse(source, buf_size=65536, **config)
Docstring: <no docstring>
File:      ~/venvs/work/lib/python3.12/site-packages/ijson/common.py
Type:      function